Construcción de AG para encontrar el mejor despacho de energía minimizando los costos de transporte y generación.

# **Datos Iniciales**

Capacidad de las plantas: `[3, 6, 5, 4]` GW

Necesidades de las ciudades: `[4, 3, 5, 3]` GW

Costos de transporte (por GW):

In [ ]:
costos_transporte = [
    [1,4,3,6], #Planta C
    [4,1,4,5], #Planta B
    [3,4,1,4], #Planta M
    [6,5,4,1]  #Planta B
]

Costos por kW-H por generador:

In [ ]:
costos_generacion = [680,720,660,750] #Planta C, B, M, B

# **Algoritmo Genético para Optimización**

Se define el tamaño de la población, el número de generaciones y la tasa de mutación; se genera la población inicial de soluciones, se evalúa la aptitud de cada individuo (solución) basado en los costos; se seleccionan los mejores individuos para cruzar y crear la siguiente generación, se aplican mutación para mantener la diversidad genética; y, se repeta hasta alcanzar el criterio de parada.

In [ ]:
import numpy as np
import random

capacidad_plantas = [3,6,5,4]
necesidades_ciudades = [4,3,5,3]
costos_transporte = [
    [1,4,3,6], #Planta C
    [4,1,4,5], #Planta B
    [3,4,1,4], #Planta M
    [6,5,4,1]  #Planta B
]
costos_generacion = [680, 720, 660, 750]  #Planta C,B,M,B

def generar_poblacion(tam_poblacion, num_plantas, num_ciudades):
    return [np.random.permutation(num_plantas * num_ciudades).reshape(num_plantas, num_ciudades) for i in range(tam_poblacion)]

def evaluar_aptitud(individuo):
    total_costo = 0
    for i in range(len(capacidad_plantas)):
        for j in range(len(necesidades_ciudades)):
            if individuo[i][j] > 0:
                costo_transporte = costos_transporte[i][j] * individuo[i][j]
                costo_generacion = costos_generacion[i] * individuo[i][j]
                total_costo += costo_transporte + costo_generacion
    return total_costo

def seleccionar_padres(poblacion, aptitudes, num_padres):
    return random.choices(poblacion, weights=aptitudes, k=num_padres)

def cruzar_padres(padre1, padre2):
    punto_cruce = random.randint(1, padre1.size - 1)
    hijo1 = np.concatenate((padre1.flatten()[:punto_cruce], padre2.flatten()[punto_cruce:])).reshape(padre1.shape)
    hijo2 = np.concatenate((padre2.flatten()[:punto_cruce], padre1.flatten()[punto_cruce:])).reshape(padre1.shape)
    return hijo1,hijo2

def mutar_individuo(individuo, tasa_mutacion):
    for i in range(individuo.shape[0]):
        for j in range(individuo.shape[1]):
            if random.random() < tasa_mutacion:
                individuo[i][j] = random.randint(0, max(capacidad_plantas[i], necesidades_ciudades[j]))
    return individuo

def algoritmo_genetico(tam_poblacion, num_generaciones, tasa_mutacion):
    num_plantas = len(capacidad_plantas)
    num_ciudades = len(necesidades_ciudades)
    poblacion = generar_poblacion(tam_poblacion, num_plantas, num_ciudades)
    mejores_aptitudes = []

    for _ in range(num_generaciones):
        aptitudes = [evaluar_aptitud(individuo) for individuo in poblacion]
        mejores_aptitudes.append(min(aptitudes))
        padres = seleccionar_padres(poblacion, aptitudes, tam_poblacion // 2)
        nuevos_individuos = []

        for i in range(0, len(padres), 2):
            padre1, padre2 = padres[i], padres[i + 1]
            hijo1, hijo2 = cruzar_padres(padre1, padre2)
            nuevos_individuos.extend([hijo1, hijo2])

        poblacion = [mutar_individuo(individuo, tasa_mutacion) for individuo in nuevos_individuos]

    mejor_individuo = min(poblacion, key=lambda ind: evaluar_aptitud(ind))
    return mejor_individuo, mejores_aptitudes

tam_poblacion=100
num_generaciones=1000
tasa_mutacion=0.01

mejor_individuo,mejores_aptitudes = algoritmo_genetico(tam_poblacion, num_generaciones, tasa_mutacion)

print("Mejor Individuo (Distribución de Energía):")
print(mejor_individuo)
print("Mejor Aptitud (Costo Mínimo):",min(mejores_aptitudes))